# EV353 Problem Set 3 - Coupled Climate Variability and Buoyancy Forcing 
### Part 2: Python Data Analysis

### Overview

Ocean-atmosphere interactions give rise to many flavors of natural climate variability that have significant impacts on society, including freshwater resources, human health, and hurricane activity. The ocean’s density-driven thermohaline circulation also helps shape global climate, particularly on longer timescales ranging from decades to millennia.

In this problem set, you will demonstrate both your qualitative and quantitative understanding of the mechanisms by which the ocean and atmosphere influence each other and how the ocean responds to buoyancy forcing. Your responses should incorporate the concepts covered in class and the assigned readings. Chapters 6.3, 7 and 8 in Physical Oceanography and Climate by Kris Karnauskas and this week’s scientific papers will be helpful references. 

**To foster a collaborative learning environment, you will complete this lab with a partner.** The pairings will be randomly determined in class and posted on Canvas. Each person is required to complete and contribute to all questions. This includes conducting all calculations/analyses and writing your own code. *It is against the honor code to divide up the questions among different individuals or use generative AI/chatbots on any aspect this assignment. All answers must be written in your own words.*


### Learning Outcomes

After completing this assignment, you will be able to:
- Describe the mechanisms by which the ocean and atmosphere influence each other to yield modes of natural climate variability. Here focusing on the El Niño-Southern Oscillation (ENSO).
- Characterize observed ENSO variability using real-world observations.
- Compare and contrast the dominant physical processes leading to deep water formation.
- Identify and articulate the significance of ENSO and Meridional Overturning Circulation in the context of the broader global climate system.

### Assignment Structure

**Part 1** includes written responses and hand calculations. To receive full credit, you must show your work and explain your reasoning. **Part 2** includes data analysis exercises in Python. You will use this Jupyter Notebooks to manipulate and visualize geospatial datasets, perform calculations, and interpret the results.

---
### Data Sets 
To complete this lab, you will need the following data sets (posted on Canvas):
1. ``oiv2.nc`` NOAA Optimal Interpolation Sea Surface Temperature version 2 (OISST v2) 
2. ``rapid.nc`` RAPID array of moorings to measure the variability of the meridional overturning circulation (MOC)
3. `ersst.nc` NOAA Extended Reconstructed SST V5

---
### Assessment
Generating high-quality figures is an important part of science comunication! All figures (time series & maps) must be properly formatted to receive full credit. This means including titles, axes labels (with units), and legends, as appropriate. The aspect ratio should be appropriate for the type of figure (i.e., the figures are not compressed or stretched). 

Please use the Python packages [cartopy](https://scitools.org.uk/cartopy/docs/latest/) and [cmocean colormaps for oceanography](https://matplotlib.org/cmocean/) when generating maps. Take care to use intuitive colormaps and colorbar levels for the type of data you are representing. 

The grading rubric for part 2 is provided at the end of the companion Word Document for this assignment.

In [1]:
# import Python packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import scipy.stats

### Question 1 [13 points]: Global SST Variability and ENSO Indices

Using the NOAA OISSTv2 product, calculate the four common ENSO indices based on monthly SST anomalies: NINO1+2, NINO3, NINO3.4, and NINO4 (a map is provided in the Word document for the problem set). Remember to remove the average seasonal cycle (i.e., the climatology) to calculate monthly SST anomalies. Explore their variability by making a figure that overlays these four time series. Add a title, legend, and label all axes (including the units). Include a horizontal line at 0°C for reference. Paste your plot in your main problem set document. 

**a.** Both qualitatively and quantitatively, how do the four indices compare to one another in terms of their amplitude of SST variability (e.g., standard deviation)? 

**b.** Generate a global map of some measure of the variability of SST anomalies. Given that there is no analysis over land, fill in the land to highlight only the SST data (e.g., with a light gray color).Include your map in this document. Does this map confirm your results from part (a)?

**c.** Which two ENSO indices have the lowest temporal correlation with one another? To answer this question, calculate the Pearson correlation coefficients for different combinations of indices and fill in the unshaded cells in the table provided in the problem set document.

**d.**	Define a criterion (or set of criteria) for an El Niño event. Based on this criterion, how many El Niño events occurred from 1982 to the end of the dataset? What period (as in 1/frequency) does that imply for ENSO as an oscillation? Does your simple estimate of the period fall within the widely-cited ENSO period of 2-7 years?

**e.**	Google the latest ENSO forecast from the NOAA Climate Prediction Center (CPC) or the International Research Institute for Climate and Society (IRI). If there is a forecasted El Niño or La Niña on the horizon, how does it compare to past events? Would it qualify as an El Niño based on your definition in part (d) of this question?

---
**Hints:**

*Calculating the ENSO Indices*
- Use the xarray `DataArray.groupby()` method to group the data by month and calculate the monthly means for every grid point in the data (e.g., average January, average February, etc...). Here `DataArray` would be subsituted with the name of your DataArray variable. This is the average seasonal cycle.
- For example: `DataArray.groupby(DataArray['time'].dt.month)` (see example code below) will give you a DataArray grouped over 'month.' Use this to determine the average (i.e., the mean) seasonal cycle.
- Subtract the average seasonal cycle from the SST values to calculate the monthly SST anomalies at each grid point.
- The OISSTv2 data set is on a [0, 360] E longitude grid. If a location is in the western hemisphere, add 360 to the longitude values (expressed as negative numbers) to convert from degrees W to degrees E.
- Use the xarray `DataArray.where()` function to subset the data to a specified region based on latitude/longitude (an example for the tropical Pacific is below).

*Calculating Pearson correlation coefficients*
- Use the SciPy Stats Python package to calculate the Pearson correlation coefficeints: `scipy.stats.pearsonr(x,y)` where x and y are the two times series you want to correlate.

**Example code:**

```
# group a DataArray called 'sst' by month and calculate the average monthly values (i.e., the climatology)
sst.groupby(sst['time'].dt.month).mean()

# calculate average SST for the whole tropical Pacific (here sst is a DataArray)
tropicalPacific = sst.where((ssta.lat >= -30) & (ssta.lat <= 30) & 
                     (sst.lon >= 120) & (sst.lon <= 280), 
                     drop = True).mean(dim=['lat','lon'])
```

### Question 2 [12 points]: AMOC Variability
This question uses both the RAPID array and the NOAA ERSSTv5 data products.

**a.**	Using the RAPID 26°N array, plot the time series of 12-hour AMOC transport (variable: ‘moc_mar_hc10’). Overlay the 90-day smoothed AMOC transport (variable: ‘moc_mar_hc10_smooth90’) as a thicker line. Label all axes with appropriate units and add a title and legend. Include your plot in this document. 

**b.**	In what month and year did the strongest negative departure occur in the 90-day-smoothed AMOC transport time series? 

**c.** Using the NOAA ERSSTv5 dataset, calculate the climatology-removed SST anomalies. Then produce a global map of the SST anomalies during the month with the strongest negative AMOC departure determed in (b). Plot a horizontal line at 26.5°N representing the location of the RAPID array. Include your formatted map in this document.

**d.** Comment on the pattern of SST anomaly in the Atlantic basin, especially relative to the RAPID Array measurements (i.e., how do the SST anomalies compare north and south of 26.5°N?)

---
**Hints:**
- The `DataArray.argmin()` function will return the index of the minimum value of a DataArray
- This index can then be applied to the ERSSTv5 data

**Example code:**
```
# index for the minimum AMOC transport value (here for a DataArray called 'amoc')
minIdx = amoc.argmin()

# the time that corresponds with the the minimum AMOC transport value
timeMinAMOC = amoc['time'][minIdx]
```

### Canvas Submission

Work closely with your partner to ensure your answers reflect your joint understanding of the material. 

**To submit your lab:**
1. Re-run the whole Notebook and check there are no errors
2. Submit your Jupyter Notebook (LastName_FirstName_EV353_PS3.ipynb) on Canvas (1 submission per person).
3. Compile your group's written responses and all the figures you generated in a separate Word or Google document. Export the document as a PDF and upload it to Canvas (1 submission per group).